In [6]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import joblib
import pandas as pd
import math
import mysql.connector

# Load the saved model
model = tf.keras.models.load_model('model.h5')

# Load the scaler
scaler_filename = 'scaler.save'
scaler = joblib.load(scaler_filename)

# Load the dataset with pandas
dataset = pd.read_csv('dataset.csv')

In [20]:
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth's radius in meters

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Calculate differences between latitudes and longitudes
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Apply Haversine formula
    a = math.sin(delta_lat/2) * math.sin(delta_lat/2) + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon/2) * math.sin(delta_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c

    return distance

In [15]:
def getdata(category):
    
    mydb = mysql.connector.connect(
        host="34.101.230.69",
        user="root",
        password="CapstoneBisaYuk@123",
        database="my_database"
    )

    # Create a cursor
    mycursor = mydb.cursor()

    # Make a query that gets all the data from the table 'TOKO' with value of kategori = category
    sql = "SELECT * FROM TOKO WHERE kategori = %s"

    # Execute the query
    mycursor.execute(sql, (category,))

    # Fetch all the data
    myresult = mycursor.fetchall()

    # Put the data into a dataframe
    df = pd.DataFrame(myresult, columns=['id_toko', 'nama_toko', 'kategori', 'latitude', 'longitude', 'deskripsi', 'url_image', 'rerata_rating', 'jumlah_rating'])

    # Select the columns that will be used for the prediction
    df = df[['id_toko', 'latitude', 'longitude', 'rerata_rating', 'jumlah_rating']]

    # Close the connection
    mydb.close()

    return df


In [22]:
def predict(scaler, model, category, latitude, longitude):

    # Get the features
    features = getdata(category)

    # Create a new column for distance
    features['jarak'] = features.apply(lambda row: calculate_distance(latitude, longitude, row['latitude'], row['longitude']), axis=1)

    # Drop the latitude and longitude columns and move jarak to after nama_toko
    features = features.drop(['latitude', 'longitude'], axis=1)
    features = features[['id_toko', 'jarak', 'rerata_rating', 'jumlah_rating']]

    # Drop and save the nama_toko column, jarak, rerata_rating, and jumlah_rating columns to a variable
    dropped_features = features[['id_toko', 'jarak', 'rerata_rating', 'jumlah_rating']]
    features = features.drop(['id_toko'], axis=1)

    # Rename the columns to distance(meters), rating_overall, rating_count
    features = features.rename(columns={'jarak': 'distance(meters)', 'rerata_rating': 'rating_overall', 'jumlah_rating': 'rating_count'})

    # Normalize the features
    features_scaled = scaler.transform(features)

    # Make predictions, return them as a dataframe with the nama_toko column and the predictions column, avoiding NaN values for nama_toko
    predictions = pd.DataFrame(model.predict(features_scaled), columns=['predictions'])
    predictions = pd.concat([dropped_features.reset_index(drop=True), predictions], axis=1)

    # Reformat the predictions column to 2 decimal places
    predictions['predictions'] = predictions['predictions'].apply(lambda x: round(x, 2))

    # Sort the predictions by the predictions column in descending order
    predictions = predictions.sort_values(by=['predictions'], ascending=False)

    # Get the 3 highest predictions and return them as a list of floats
    predictions = predictions['predictions'].head(3).tolist()

    return predictions

    

In [23]:
# Set the category, domisile, latitude, and longitude
category = 'emas'
latitude = -6.221877
longitude = 106.846261

# Make predictions
predictions = predict(scaler, model, category, latitude, longitude)

# Print the predictions
print(predictions)

2/2 [==============================] - 0s 2ms/step
[0.98, 0.96, 0.96]
